In [1]:
import numpy as np
from openmm.app import *
from openmm.unit import *
from openmm import *
from openmm.app.modeller import Modeller
from pdbfixer import PDBFixer
import openmmtools

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
raw = PDBFixer('deca56D.pdb')
raw.findMissingResidues()
raw.findMissingAtoms()
raw.addMissingAtoms()

In [3]:
forcefield = ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3p.xml')
mdl = Modeller(raw.topology, raw.positions)
mdl.addHydrogens(forcefield, pH=7.0)
mdl.addSolvent(forcefield, 'tip3p', padding=1*nanometers)

PDBFile.writeFile(mdl.topology, mdl.positions, open('deca56D_solvated.pdb', 'w'))

In [4]:
system = forcefield.createSystem(mdl.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer)

integrator = LangevinMiddleIntegrator(293*kelvin,
                                      1.0/picoseconds,
                                      2.0*femtoseconds)

platform = openmm.Platform.getPlatformByName('CUDA')
platform_properties = {'DeviceIndex':'2'}
simulation = Simulation(mdl.topology, system, integrator, platform=platform,
                        platformProperties=platform_properties)
simulation.context.setPositions(mdl.positions)
simulation.minimizeEnergy()

# Trajectory saved every 2 ps
simulation.reporters.append(PDBReporter('NVT_equilibration.pdb', 1000))
simulation.reporters.append(CheckpointReporter('NVT_equilibrated.chk', reportInterval=1000))
simulation.reporters.append(StateDataReporter('NVT_file.txt', 1000, step=True, potentialEnergy=True, temperature=True))

# NVT equilibration for 100 ps 
simulation.step(50000)